In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sq
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sj_df = pd.read_csv('s3://508-team4/data/sj_compensation/sj_compensation.csv')
sj_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71946 entries, 0 to 71945
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                71946 non-null  object 
 1   department          71946 non-null  object 
 2   job_title           71946 non-null  object 
 3   total_cash          71946 non-null  float64
 4   base_salary         71247 non-null  float64
 5   overtime            38998 non-null  float64
 6   health              52959 non-null  float64
 7   retired             7404 non-null   object 
 8   year                71946 non-null  int64  
 9   city_id             71946 non-null  int64  
 10  irregular_cash      71946 non-null  float64
 11  retirement          71946 non-null  float64
 12  other_benefits      71946 non-null  float64
 13  total_benefits      71946 non-null  float64
 14  total_compensation  71946 non-null  float64
dtypes: float64(9), int64(2), object(4)
memory usage: 8.2+

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
sj_df['retired'].unique()

array(['Yes', 'No', nan], dtype=object)

In [5]:
sj_df[sj_df['city_id'].isnull()]

,name,department,job_title,total_cash,base_salary,overtime,health,retired,year,city_id,irregular_cash,retirement,other_benefits,total_benefits,total_compensation


In [6]:
row = sj_df[sj_df['job_title'] == 'BdComm Mbr']
#row

In [7]:
col_names_sj = sj_df.columns.tolist()
col_names_sj

['name',
 'department',
 'job_title',
 'total_cash',
 'base_salary',
 'overtime',
 'health',
 'retired',
 'year',
 'city_id',
 'irregular_cash',
 'retirement',
 'other_benefits',
 'total_benefits',
 'total_compensation']

In [8]:
sf_df = pd.read_csv('s3://508-team4/data/sf_compensation/sf_compensation.csv')
#sf_df['city_id'] = 2
sf_df.head()

,organization_group_code,job_family_code,job_code,year_type,year,organization_group,department_code,department,union_code,union,...,salaries,overtime,other_salaries,total_salary,retirement,health_and_dental,other_benefits,total_benefits,total_compensation,city_id
0,1,1400,1404,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,790.0,SEIU Local 1021 Misc,...,60720.01,0.00,0.00,60720.01,13653.20,14733.76,4904.34,33291.30,94011.31,2
1,1,9700,9703,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,SEIU Local 1021 Misc,...,91677.00,0.00,0.00,91677.00,17524.20,14733.76,7411.13,39669.09,131346.09,2
2,1,2900,2918,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,SEIU Local 1021 Misc,...,89106.03,0.00,1540.00,90646.03,17327.20,14733.76,7401.92,39462.88,130108.91,2
3,1,2900,2918,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,SEIU Local 1021 Misc,...,85581.11,3355.94,337.75,89274.80,16359.16,14151.56,7096.21,37606.93,126881.73,2
4,1,2900,2905,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,SEIU Local 1021 Misc,...,86457.00,0.00,2090.00,88547.00,16925.97,14733.76,7257.89,38917.62,127464.62,2


In [9]:
sf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799562 entries, 0 to 799561
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   organization_group_code  799562 non-null  int64  
 1   job_family_code          799562 non-null  object 
 2   job_code                 799562 non-null  object 
 3   year_type                799562 non-null  object 
 4   year                     799562 non-null  int64  
 5   organization_group       799562 non-null  object 
 6   department_code          799560 non-null  object 
 7   department               799560 non-null  object 
 8   union_code               799383 non-null  float64
 9   union                    799383 non-null  object 
 10  job_family               799562 non-null  object 
 11  job                      799557 non-null  object 
 12  employee_identifier      799562 non-null  int64  
 13  salaries                 799562 non-null  float64
 14  over

In [10]:
col_names_sf = sf_df.columns.tolist()
col_names_sf

['organization_group_code',
 'job_family_code',
 'job_code',
 'year_type',
 'year',
 'organization_group',
 'department_code',
 'department',
 'union_code',
 'union',
 'job_family',
 'job',
 'employee_identifier',
 'salaries',
 'overtime',
 'other_salaries',
 'total_salary',
 'retirement',
 'health_and_dental',
 'other_benefits',
 'total_benefits',
 'total_compensation',
 'city_id']

In [11]:
la_df = pd.read_csv('s3://508-team4/data/la_compensation/la_compensation.csv')
#la_df['city_id'] = 3
la_df.info()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (0,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753959 entries, 0 to 753958
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   record_nbr          753959 non-null  object 
 1   year                753959 non-null  int64  
 2   department_no       753959 non-null  int64  
 3   department          753959 non-null  object 
 4   job_class_pgrade    753415 non-null  object 
 5   job_title           753415 non-null  object 
 6   employment_type     753959 non-null  object 
 7   job_status          753959 non-null  object 
 8   mou                 753273 non-null  object 
 9   mou_title           753166 non-null  object 
 10  base_salary         753959 non-null  float64
 11  overtime            753525 non-null  float64
 12  irregular_cash      753525 non-null  float64
 13  total_cash          753959 non-null  float64
 14  retirement          753959 non-null  float64
 15  health              753959 non-nul

In [12]:
la_df.head()

,record_nbr,year,department_no,department,job_class_pgrade,job_title,employment_type,job_status,mou,mou_title,...,irregular_cash,total_cash,retirement,health,gender,ethnicity,total_benefits,total_compensation,other_benefits,city_id
0,303030303632,2017,98,WATER AND POWER,3156-5,CUSTODIAN,FULL_TIME,ACTIVE,8,OPERATING MAINTENANCE AND SERVICE UNIT,...,2021.84,62532.13,3678.0,23508.9,FEMALE,HISPANIC,27186.9,89719.03,NaN,3
1,3030303036,2017,98,WATER AND POWER,9105-5,UTILITY ADMINISTRATOR,FULL_TIME,ACTIVE,M,MANAGEMENT EMPLOYEES UNIT,...,6170.49,161685.87,9186.0,23508.9,FEMALE,ASIAN AMERICAN,32694.9,194380.77,NaN,3
2,303030313232,2017,98,WATER AND POWER,9602-4,WATER SERVICES MANAGER,FULL_TIME,ACTIVE,M,MANAGEMENT EMPLOYEES UNIT,...,12504.30,258383.42,16228.0,23508.9,MALE,BLACK,39736.9,298120.32,NaN,3
3,303030313632,2017,98,WATER AND POWER,5885-5,WTR TRTMT OPR,FULL_TIME,ACTIVE,6,STEAM PLANT AND WATER SUPPLY UNIT,...,12630.52,121949.85,6699.0,23508.9,MALE,ASIAN AMERICAN,30207.9,152157.75,NaN,3
4,303030323632,2017,98,WATER AND POWER,3841-5,ELTL MCHC,FULL_TIME,ACTIVE,8,OPERATING MAINTENANCE AND SERVICE UNIT,...,1566.75,125196.24,6689.0,23508.9,MALE,HISPANIC,30197.9,155394.14,NaN,3


In [13]:
la_df[la_df['base_salary'].isnull()]

,record_nbr,year,department_no,department,job_class_pgrade,job_title,employment_type,job_status,mou,mou_title,...,irregular_cash,total_cash,retirement,health,gender,ethnicity,total_benefits,total_compensation,other_benefits,city_id


In [14]:
row = la_df[la_df['city_id'] == 240894]
row

,record_nbr,year,department_no,department,job_class_pgrade,job_title,employment_type,job_status,mou,mou_title,...,irregular_cash,total_cash,retirement,health,gender,ethnicity,total_benefits,total_compensation,other_benefits,city_id


In [15]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [16]:
ingest_create_athena_db_passed = False

In [17]:
sj_df['city_id'].value_counts()

1    71946
Name: city_id, dtype: int64

## Import PyAthena

In [18]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

## Create Athena Database

In [19]:
database_name = "compens"

In [20]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [21]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [22]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS compens


In [23]:
pd.read_sql(statement, conn)

""


## Verify The Database Has Been Created Succesfully

In [24]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,compens
1,default
2,dsoaws


## Create Tables 

### San Jose Table

In [25]:
#Directory for the data input
data_dir = 's3://508-team4/data'

In [26]:
table_name1='sj_compensation'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name1}', conn)


create_table = f""" 
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name1}(
                name string, 
                department string, 
                job_title string, 
                total_cash float,  
                base_salary float, 
                overtime float, 
                health float, 
                retired string,
                year int,
                city_id int,
                irregular_cash float, 
                retirement float, 
                other_benefits float,
                total_benefits float,
                total_compensation float
 )
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{data_dir}/{table_name1}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

pd.read_sql(f'SELECT * FROM {database_name}.{table_name1} LIMIT 5', conn)
                

,name,department,job_title,total_cash,base_salary,overtime,health,retired,year,city_id,irregular_cash,retirement,other_benefits,total_benefits,total_compensation
0,Bustillos Steven D,Police,Police Sergeant,286137.70,90888.00,89867.88,13640.50,Yes,2013,1,105381.81,74429.71,0.0,88070.21,374207.9
1,Figone Debra J,City Manager,City Manager U,248564.84,227975.02,NaN,15166.00,Yes,2013,1,20589.82,151550.08,0.0,166716.08,415280.9
2,Guerra Daniel P,Police,Police Officer,241039.12,97198.40,132104.55,15371.56,No,2013,1,11736.18,79821.51,0.0,95193.07,336232.2
3,Moore Christopher M,Police,Chief Of Police U,233540.31,15319.54,NaN,1371.81,Yes,2013,1,218220.78,11204.16,0.0,12575.97,246116.3
4,Vasquez Hector M,Police,Police Officer,230469.84,97198.40,124552.69,16345.32,No,2013,1,8718.76,79821.51,0.0,96166.83,326636.7


In [27]:
# pd.read_sql(f"SELECT * FROM {database_name}.{table_name2} WHERE city_id = 1", conn)

### San Francisco Table

In [28]:
table_name2 ='sf_compensation'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name2}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name2}(
                organization_group_code int, 
                job_family_code string,
                job_code string, 
                year_type string, 
                year int, 
                organization_group string, 
                department_code string, 
                department string, 
                union_code float, 
                union string, 
                job_family string, 
                job_title string, 
                employee_identifier int, 
                base_salary float, 
                overtime float, 
                irregular_cash float, 
                total_cash float, 
                retirement float,
                health float, 
                other_benefits float, 
                total_benefits float, 
                total_compensation float,
                city_id int
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{data_dir}/{table_name2}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

pd.read_sql(f'SELECT * FROM {database_name}.{table_name2} LIMIT 5', conn)

,organization_group_code,job_family_code,job_code,year_type,year,organization_group,department_code,department,union_code,union,...,base_salary,overtime,irregular_cash,total_cash,retirement,health,other_benefits,total_benefits,total_compensation,city_id
0,1,Q000,Q004,Fiscal,2019,Public Protection,POL,Police,911.0,POA,...,129681.14,6167.59,9093.34,144942.06,24200.20,15525.74,2460.53,42186.47,187128.55,2
1,1,Q000,Q052,Fiscal,2019,Public Protection,POL,Police,911.0,POA,...,147605.60,10702.01,6672.25,164979.84,26206.21,16606.20,2769.44,45581.85,210561.70,2
2,1,1000,1094,Fiscal,2019,Public Protection,POL,Police,21.0,Prof & Tech Eng Local 21,...,122011.03,0.00,9016.07,131027.10,22725.35,14628.56,10355.33,47709.24,178736.34,2
3,1,8200,8260,Fiscal,2019,Public Protection,POL,Police,21.0,Prof & Tech Eng Local 21,...,116513.83,0.00,0.00,116513.83,22271.87,14628.56,9426.76,46327.19,162841.02,2
4,1,9200,9209,Fiscal,2019,Public Protection,POL,Police,790.0,SEIU Local 1021 Misc,...,75927.11,8322.75,2240.53,86490.39,14941.89,14691.25,6964.87,36598.01,123088.40,2


### Los Angeles Table

In [29]:
table_name3 ='la_compensation'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name3}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name3}(
                record_nbr string,
                year int,
                department_no string,
                department string,
                job_class_pgrade int,
                job_title string,
                employment_type string,
                job_status string,
                mou string,
                mou_title string,
                base_salary float,
                overtime float,
                irregular_cash float,
                total_cash float,
                retirement float,
                health float,
                gender string,
                ethnicity string,
                total_benefits float,
                total_compensation float,
                other_benefits float,
                city_id int
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{data_dir}/{table_name3}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

pd.read_sql(f'SELECT * FROM {database_name}.{table_name3} LIMIT 5', conn)
                

,record_nbr,year,department_no,department,job_class_pgrade,job_title,employment_type,job_status,mou,mou_title,...,irregular_cash,total_cash,retirement,health,gender,ethnicity,total_benefits,total_compensation,other_benefits,city_id
0,323931383034,2019,88,RECREATION AND PARKS,None,RECREATION INSTRUCTOR,PART_TIME,ACTIVE,07,RECREATION ASSISTANTS,...,0.00,863.52,0.00,0.00,FEMALE,OTHER,0.00,863.52,NaN,3
1,3239313832,2019,40,GENERAL SERVICES,None,EQUIPMENT MECHANIC,FULL_TIME,ACTIVE,14,SERVICE AND CRAFT,...,1151.47,102172.61,27424.33,19731.84,MALE,ASIAN,47156.17,149328.78,NaN,3
2,3239313833,2019,38,FIRE,None,FIRE CAPTAIN I,FULL_TIME,ACTIVE,23,FIREFIGHTERS AND FIRE CAPTAINS,...,3898.78,260528.44,66620.33,18556.18,MALE,HISPANIC,85176.51,345704.94,NaN,3
3,3239313835,2019,82,PUBLIC WORKS - SANITATION,None,REFUSE COLLECTION TRUCK OPERATOR II,FULL_TIME,ACTIVE,04,EQUIPMENT OPERATION AND LABOR,...,1110.00,84796.94,22990.53,19731.84,MALE,HISPANIC,42722.37,127519.31,NaN,3
4,3239313837,2019,38,FIRE,None,FIRE CAPTAIN I,FULL_TIME,ACTIVE,23,FIREFIGHTERS AND FIRE CAPTAINS,...,5454.48,330640.90,64629.95,18556.18,MALE,CAUCASIAN,83186.13,413827.03,NaN,3


In [30]:
# cursor = conn.cursor()
# cursor.execute(f'''ALTER TABLE {database_name}.{table_name3} ADD COLUMNS (other_benefits FLOAT)''')
# conn.commit()
# cursor.close()
# pd.read_sql(f'SELECT * FROM {database_name}.{table_name3} LIMIT 5', conn)

In [31]:
# change_table = f'''ALTER TABLE {database_name}.{table_name1} CHANGE (pay_year year int, department_title department string)'''
# cursor.execute(change_table)
# conn.commit()
# # pd.read_sql(f'SELECT * FROM {database_name}.{table_name1} LIMIT 5', conn)

In [32]:
# cursor = conn.cursor()

# cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE pay_year year int')
# cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE department_title department string')
# cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE regular_pay base_salary float')
# cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE overtime_pay overtime float')
# cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE all_other_pay irregular_cash float')
# cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE total_pay total_cash float')
# cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE city_retirement_contributions retirement float')
# cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE benefit_pay health float')

# conn.commit()
  
# pd.read_sql(f'SELECT * FROM {database_name}.{table_name1} LIMIT 5', conn)
   

## Merge the compensation tables

In [33]:
# #The 3 table were merge based on the columns of interest
# df = pd.read_sql(f'SELECT year, department, job_title, base_salary, overtime, irregular_cash, \
#             total_cash, retirement, health, other_benefits, \
#             total_benefits, total_compensation, city_id \
#         FROM {database_name}.{table_name1} \
#         UNION ALL SELECT year, department, job_title, base_salary, overtime, irregular_cash, \
#             total_cash, retirement, health, other_benefits, total_benefits, total_compensation, city_id \
#         FROM {database_name}.{table_name2} \
#         UNION ALL SELECT year, department, job_title, base_salary, overtime, irregular_cash, \
#             total_cash, retirement, health, other_benefits, \
#             total_benefits, total_compensation, city_id \
#       FROM {database_name}.{table_name3}', conn)

In [34]:
# pd.set_option('display.max_columns', 500)
# null_test = pd.read_sql(f"SELECT * FROM {database_name}.{table_name3} WHERE city_id IS NULL", conn)
# null_test

In [35]:
# pd.set_option('display.max_columns', 500)
# null_test = pd.read_sql(f"SELECT * FROM {database_name}.{table_name1} WHERE overtime IS NULL", conn)

In [36]:
#df.to_csv('compensation.csv')

In [37]:
 #df.info()

In [38]:
# df.info()

### cpi table

In [39]:
table_name4 ='cpi'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name4}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name4}(\
                year int, \
                annual_average_cpi float, \
                inflation_rate float)
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{data_dir}/{table_name4}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

pd.read_sql(f'SELECT * FROM {database_name}.{table_name4} LIMIT 5', conn)
                

,year,annual_average_cpi,inflation_rate
0,2013,233.0,1.5
1,2014,236.7,1.6
2,2015,237.0,0.1
3,2016,240.0,1.3
4,2017,245.1,2.1


In [40]:
#Check tables in the database
df_show = pd.read_sql('SHOW TABLES IN compens', conn)
df_show.head(5)


,tab_name
0,cpi
1,la_compensation
2,sf_compensation
3,sj_compensation


## Merge the compensation and cpi tables

In [41]:
# The 3 table were merge based on the columns of interest
df2 = pd.read_sql(f'SELECT t.*, t4.annual_average_cpi, t4.inflation_rate \
        FROM (SELECT year, department, job_title, base_salary, overtime, irregular_cash, \
            total_cash, retirement, health, other_benefits, total_benefits, total_compensation, city_id \
        FROM {database_name}.{table_name1} \
        UNION ALL SELECT year, department, job_title, base_salary, overtime, irregular_cash, \
            total_cash, retirement, health, other_benefits, total_benefits, total_compensation, city_id \
        FROM  {database_name}.{table_name2} \
        UNION ALL SELECT year, department, job_title, base_salary, overtime, irregular_cash, \
            total_cash, retirement, health, other_benefits, total_benefits, total_compensation, city_id \
        FROM  {database_name}.{table_name3}) t \
        JOIN {database_name}.{table_name4} t4 ON t.year = t4.year', conn)


In [42]:
df2.tail()

,year,department,job_title,base_salary,overtime,irregular_cash,total_cash,retirement,health,other_benefits,total_benefits,total_compensation,city_id,annual_average_cpi,inflation_rate
1466584,2015,PUBLIC WORKS - ENGINEERING,SENIOR STRUCTURE ENGINEER PROJECT MANAGER II,144622.40,0.00,938.66,145561.06,42895.00,16276.56,NaN,59171.56,204732.62,3,237.0,0.1
1466585,2015,FIRE,FIREFIGHTER III,113449.48,48316.87,32383.08,194149.44,49351.81,15373.30,NaN,64725.11,258874.55,3,237.0,0.1
1466586,2015,POLICE,POLICE OFFICER II,70390.92,5737.12,3678.60,79806.64,32978.15,15230.65,NaN,48208.80,128015.44,3,237.0,0.1
1466587,2015,POLICE,POLICE OFFICER III,100380.80,0.00,2357.00,102737.80,47028.40,15317.75,NaN,62346.15,165083.95,3,237.0,0.1
1466588,2015,ZOO,RECREATION ASSISTANT,5863.95,0.00,0.00,5863.95,0.00,0.00,NaN,0.00,5863.95,3,237.0,0.1


In [43]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1466589 entries, 0 to 1466588
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   year                1466589 non-null  int64  
 1   department          1466589 non-null  object 
 2   job_title           1466589 non-null  object 
 3   base_salary         1465890 non-null  float64
 4   overtime            1433207 non-null  float64
 5   irregular_cash      1466155 non-null  float64
 6   total_cash          1466589 non-null  float64
 7   retirement          1466589 non-null  float64
 8   health              1447602 non-null  float64
 9   other_benefits      830557 non-null   float64
 10  total_benefits      1466589 non-null  float64
 11  total_compensation  1466589 non-null  float64
 12  city_id             1466589 non-null  int64  
 13  annual_average_cpi  1466589 non-null  float64
 14  inflation_rate      1466589 non-null  float64
dtypes: float64(11),

In [44]:
#df2.to_csv('compensation_cpi.csv')

In [46]:
df2.to_csv('compensation_cpi.csv.gz', index=False, compression='gzip')

In [ ]:
#conn.close()

# EDA

In [ ]:
#Count the number of duplicates
num_duplicates = df2.duplicated().sum()

num_duplicates

In [ ]:
# get number of rows and columns
print('Number of Rows:', df2.shape[0])
print('Number of Columns:', df2.shape[1], '\n')

# inspect datatypes and nulls
data_types = df2.dtypes
data_types = pd.DataFrame(data_types)
data_types = data_types.assign(Null_Values = 
                               df2.isnull().sum())
data_types.reset_index(inplace = True)
data_types.rename(columns={0:'Data Type',
                          'index': 'Column/Variable',
                          'Null_Values': "# of Nulls"})

In [ ]:
df2['city_id'].unique()

In [ ]:
# row = df2[df2['city_id'] == 1]
# row

In [ ]:
df2[df2['city_id'].isnull()].info()

In [ ]:
df2.info()

In [ ]:
df2['year'].unique()

In [ ]:
# %%html

# <p><b>Shutting down your kernel for this notebook to release resources.</b></p>
# <button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
# <script>
# try {
#     els = document.getElementsByClassName("sm-command-button");
#     els[0].click();
# }
# catch(err) {
#     // NoOp
# }    
# </script>

In [ ]:
# %%javascript

# try {
#     Jupyter.notebook.save_checkpoint();
#     Jupyter.notebook.session.delete();
# }
# catch(err) {
#     // NoOp
# }